# EAS 110 Project Code

In [ ]:
"""

Please try your best to work them out independently. 

"""

## Import Module

In [105]:
import numpy as np
import random
from collections import defaultdict
import heapq

## Create the Enironment

In [140]:
# 1. Define the Default Grid World
default_world = [[1, 1, 2, 1, 2],
                 [2, 1, 7, 1, 5],
                 [3, 5, 6, 4, 4],
                 [3, 2, 3, 4, 2],
                 [2, 1, 8, 5, 0],
                 [1, 2, 6, 5, 3]]

#default_obstacles = {(2, 2), (3, 2)}  # obstacles in the grid

default_actions = [(-1, 0), (1, 0), (0, -1), (0, 1)] # actions = [up, down, left, right]

default_start=(1, 0)

default_goal=(4, 4)

## Project 1

### DP - Task 1

In [ ]:
"""
Assume that the robot can only move to right and down.
Therefore, actions = [down, right] in this case.
"""

In [141]:
world = default_world
start = default_start
goal = default_goal

In [138]:
def DP_NoRecurrsion(world, start, goal):
    #initialization
    Rs, Cs, Rg, Cg = start[0], start[1], goal[0], goal[1]
    cost = [[float('inf')] * len(world[0]) for _ in range(len(world))]
    cost[Rs][Cs] = world[Rs][Cs]
    
    for i in range(Rs + 1, Rg + 1):
        cost[i][Cs] = cost[i - 1][Cs] + world[i][Cs]
    for j in range(Cs + 1, Cg + 1):
        cost[Rs][j] = cost[Rs][j - 1] + world[Rs][j]
      
    #traverse the map
    for i in range(Rs + 1, Rg + 1):
        for j in range(Cs + 1, Cg + 1):
            cost[i][j] = min(cost[i - 1][j], cost[i][j - 1]) + world[i][j]
    
    return cost

def reconstruct_path(cost, start, goal):
    Rs, Cs, Rg, Cg = start[0], start[1], goal[0], goal[1]
    current = [goal[0], goal[1]]
    path = []
    path.append(current)
    
    while current != start:
        i, j = current[0], current[1]
        if cost[i - 1][j] < cost[i][j - 1]:
            current = (i - 1, j)
        else:
            current = (i, j - 1)
        path.append(current)  
    path.reverse()

    return path

In [142]:
#Output the Answer
cost = DP_NoRecurrsion(world, start, goal)
print("Minimun Cost: ", cost[goal[0]][goal[1]])

path = reconstruct_path(cost, start, goal)
print("Optimal Path: ", path)

Minimun Cost:  19
Optimal Path:  [(1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), [4, 4]]


In [ ]:
"""
Questions:
1. Did I waste some space in memory? Can you reduce the waste?
    Hint: cost = [[0 for _ in range(goal[0] - start[0])] for _ in range(goal[1] - start[1])]
    
2. Can you rewrite the code using recurrsion? Talk about the benefits and drawbacks of doing so.
"""

### Dijkstra - Task 2

In [ ]:
"""
This time, actions = [up, down, left, right].
"""

In [143]:
world = default_world
start = default_start
goal = default_goal

In [145]:
def dijkstra(world, start, goal):
    """
    :world: 2D list of costs, world[row][col] is the running cost.
    :start: a tuple (row, col) that represents start.
    :goal: a tuple (row, col) that represents   goal.
    :return: (min_cost, optimal path) 
    """

    # Dimensions of the world
    rows = len(world)
    cols = len(world[0])

    # Initialize the cost map
    cost = [[float('inf')] * cols for _ in range(rows)]
    # Distance to the start is the cost of the start cell itself
    cost[start[0]][start[1]] = world[start[0]][start[1]]

    # Priority queue for cells to explore; stores tuples of (cost, (row, col))
    pq = [(world[start[0]][start[1]], start)]

    # Keep track of predecessors to reconstruct path
    # predecessor[(r, c)] = (prev_r, prev_c)
    predecessor = {start: None}

    # Directions for up, down, left, right
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

    while pq:
        current_cost, (r, c) = heapq.heappop(pq)

        # If we've reached the goal, we can reconstruct the path and return
        if (r, c) == goal:
            return current_cost, reconstruct_path(predecessor, start, goal)

        # If we pop a cell whose distance doesn't match the cost array, skip it 
        # (it means this entry is outdated).
        if current_cost > cost[r][c]:
            continue

        # Check neighbors
        for dr, dc in directions:
            nr, nc = r + dr, c + dc
            # Validate neighbor is within world bounds
            if 0 <= nr < rows and 0 <= nc < cols:
                # Cost to move to neighbor is current_cost + cost of neighbor cell
                new_cost = current_cost + world[nr][nc]
                
                # If we found a cheaper path to (nr, nc), update and push to queue
                if new_cost < cost[nr][nc]:
                    cost[nr][nc] = new_cost
                    predecessor[(nr, nc)] = (r, c)
                    heapq.heappush(pq, (new_cost, (nr, nc)))

    # If goal was never reached, return infinity cost and empty path
    return float('inf'), []

def reconstruct_path(predecessor, start, goal):
    """
    Reconstruct path from predecessor dictionary.
    """
    path = []
    current = goal
    while current is not None:
        path.append(current)
        current = predecessor[current]
    path.reverse()  # reverse to get path from start to goal

    # Sanity check: if the start of the path is not the actual start, something went wrong
    if path[0] != start:
        return []
    return path

In [148]:
cost_optimal, path_optimal = dijkstra(world, start, goal)
print("Minimun Cost: ", cost_optimal)
print("Optimal Path: ", path_optimal)

Minimun Cost:  18
Optimal Path:  [(1, 0), (0, 0), (0, 1), (0, 2), (0, 3), (1, 3), (2, 3), (2, 4), (3, 4), (4, 4)]


## Project 2

In [149]:
# Define the grid-world environment
class GridWorld:
    def __init__(self, env, actions, start, goal, obstacles=None):
        """
        env: the grid-world.
        start: tuple (row, col) for the start state.
        goal: tuple (row, col) for the goal state.
        obstacles: set of (row, col) cells that are blocked.
        """
        width = len(env[0])
        height = len(env)
        nA = len(actions)
        
        self.env = env
        self.width = width
        self.height = height
        self.actions = actions 
        self.nA = nA
        
        # Make sure that start and goal are in the world!
        if start[0] < height and start[1] < width:
            self.start = start
        else:
            print("Invalid Start")
            
        if goal[0] < height and goal[1] < width:
            self.goal = goal
        else:
            print("Invalid Goal")
        
        
        # Define obstacles (if any)
        if obstacles is None:
            obstacles = set()
        self.obstacles = obstacles
        
        self.reset()  
    
    def reset(self):
        self.state = self.start
        return self.state
    
    
    def valid(self, x_new, y_new):
        if x_new < 0 or x_new > self.height - 1 or y_new < 0 or y_new > self.width - 1:
            return False
        elif self.obstacles and (x_new, y_new) in obstacles:
            return False 
        else:
            return True
    
    def step(self, action):
        x, y = self.state
        dx, dy = self.actions[action]

        if self.valid(x+dx, y+dy):
            x, y = (x+dx, y+dy) 
            
        self.state = (x, y)
        reward = self.env[x][y]  
        
        if self.state == self.goal:
            done = True
        else:
            done = False
        
        return self.state, reward, done


### SARSA

In [150]:
class SarsaAgent:
    def __init__(self, env, alpha=0.1, gamma=0.99, epsilon=0.1):
        """
        env: The GridEnv
        alpha: learning rate
        gamma: discount factor
        epsilon: exploration rate for epsilon-greedy
        """
        self.env = env
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        
        # Construct Q-table
        # It'll be overridden later
        self.q_table = {}
        
        # For convenience, let's store a list of all possible (r, c) states
        self.all_states = [
            (r, c) for r in range(env.height) 
                    for c in range(env.width) 
                    if (r, c) not in env.obstacles
        ]
        
        # Initialize Q-values to 0 for all states & actions
        for state in self.all_states:
            self.q_table[state] = np.zeros(len(env.actions))
    
    def choose_action(self, state):
        """Epsilon-greedy action selection."""
        if np.random.rand() < self.epsilon:
            # Explore
            return np.random.randint(len(self.env.actions))
        else:
            # Exploit
            return np.argmax(self.q_table[state])
    
    def update(self, state, action, reward, next_state, next_action, done):
        """Q-learning update rule."""

        current_q = self.q_table[state][action]
        
        target = reward + self.gamma * self.q_table[next_state][next_action]
        
        # SARSA-learning update
        self.q_table[state][action] = current_q + self.alpha * (target - current_q)
    
    def get_best_action(self, state):
        """Returns the best action for a given state based on current Q-values."""
        return np.argmax(self.q_table[state])


In [151]:
def train_sarsa_learning(env, agent, num_episodes=10, max_steps=50):
    rewards_history = []
    optimal_path = []
    max_reward = -100000
    
    for episode in range(num_episodes):
        #reset environment
        state = env.reset()
        #initial action
        action = agent.choose_action(state)
        
        total_reward = 0
        path = []
        
        
        for _ in range(max_steps):
            next_state, reward, done = env.step(action)
            total_reward += reward
            path.append((state, action, next_state, total_reward, agent.q_table[state][action]))        
            
            if done:
                agent.q_table[state][action] += agent.alpha * (reward - agent.q_table[state][action])
                break
                
            next_action = agent.choose_action(next_state)
            agent.update(state, action, reward, next_state, next_action, done)
            
            state = next_state
            action = next_action
                
        if total_reward > max_reward:
            max_reward = total_reward
            optimal_path = path
        
        rewards_history.append(total_reward)
    
    return rewards_history, optimal_path


In [155]:
# 1. Define the Grid World
world = default_world
start = default_start
goal = default_goal
actions = default_actions
#obstacles = {(2, 2), (3, 2)}  # Example obstacles in the grid

#CHANGE THE WORLD for minimum cost path
world = -1 * np.array(world)

env = GridWorld(world, actions, start, goal, obstacles=None)

# 2. Create Q-learning agent
agent = SarsaAgent(env, alpha=0.1, gamma=0.9, epsilon=0.8)


# 3. Train
rewards_history, path = train_sarsa_learning(env, agent, num_episodes=500, max_steps=50)

path

[((1, 0), 2, (1, 0), -2, -14.073762391247348),
 ((1, 0), 0, (0, 0), -3, -12.42987802772197),
 ((0, 0), 3, (0, 1), -4, -12.679809982667015),
 ((0, 1), 0, (0, 1), -5, -12.453910556170865),
 ((0, 1), 0, (0, 1), -6, -12.429371450609157),
 ((0, 1), 3, (0, 2), -8, -13.745388662921478),
 ((0, 2), 3, (0, 3), -9, -11.904778076794631),
 ((0, 3), 1, (1, 3), -10, -13.632720482597133),
 ((1, 3), 1, (2, 3), -14, -12.969442195727368),
 ((2, 3), 1, (3, 3), -18, -10.352193954151176),
 ((3, 3), 3, (3, 4), -20, -6.995477781015412),
 ((3, 4), 3, (3, 4), -22, -5.1510744806696565),
 ((3, 4), 1, (4, 4), -22, 0.0)]

### Q-learning

In [156]:
class QLearningAgent:
    def __init__(self, env, alpha=0.1, gamma=0.99, epsilon=0.1):
        """
        env: The GridEnv
        alpha: learning rate
        gamma: discount factor
        epsilon: exploration rate for epsilon-greedy
        """
        self.env = env
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        
        # Construct Q-table
        # It'll be overridden later
        self.q_table = {}
        
        # For convenience, let's store a list of all possible (r, c) states
        self.all_states = [
            (r, c) for r in range(env.height) 
                    for c in range(env.width) 
                    if (r, c) not in env.obstacles
        ]
        
        # Initialize Q-values to 0 for all states & actions
        for state in self.all_states:
            self.q_table[state] = np.zeros(len(env.actions))
    
    def choose_action(self, state):
        """Epsilon-greedy action selection."""
        if np.random.rand() < self.epsilon:
            # Explore
            return np.random.randint(len(self.env.actions))
        else:
            # Exploit
            return np.argmax(self.q_table[state])
    
    def update(self, state, action, reward, next_state, done):
        """Q-learning update rule."""

        current_q = self.q_table[state][action]
        
        if done:
            # No future state to consider
            target = reward
        else:
            # Max over next state's Q-values
            # Can help develop immitation learning by replaying the previous experience
            
            target = reward + self.gamma * np.max(self.q_table[next_state])
        
        # Q-learning update
        self.q_table[state][action] = current_q + self.alpha * (target - current_q)
    
    def get_best_action(self, state):
        """Returns the best action for a given state based on current Q-values."""
        return np.argmax(self.q_table[state])


In [157]:
def train_q_learning(env, agent, num_episodes=10, max_steps=50):
    rewards_history = []
    optimal_path = []
    max_reward = -100000
    
    for episode in range(num_episodes):
        state = env.reset()
        total_reward = 0
        path = []
        
        for _ in range(max_steps):
            action = agent.choose_action(state)
            next_state, reward, done = env.step(action)
            agent.update(state, action, reward, next_state, done)
            total_reward += reward
            
            path.append((state, action, next_state, total_reward, agent.q_table[state][action]))        
            state = next_state
            
            if done:
                break
                
        if total_reward > max_reward:
            max_reward = total_reward
            optimal_path = path
        
        rewards_history.append(total_reward)
    
    return rewards_history, optimal_path


In [161]:
# 1. Define the Grid World
world = default_world
start = default_start
goal = default_goal
actions = default_actions
#CHANGE THE WORLD for minimum cost path
world = -1 * np.array(world)

env = GridWorld(world, actions, start=(1, 0), goal=(4, 4), obstacles=None)

# 2. Create Q-learning agent
agent = QLearningAgent(env, alpha=0.1, gamma=0.9, epsilon=0.8)


# 3. Train
rewards_history, path = train_q_learning(env, agent, num_episodes=500, max_steps=50)

path

[((1, 0), 0, (0, 0), -1, -9.778342948357613),
 ((0, 0), 3, (0, 1), -2, -9.774189183412396),
 ((0, 1), 3, (0, 2), -4, -10.411493512515518),
 ((0, 2), 3, (0, 3), -5, -9.363153987948184),
 ((0, 3), 1, (1, 3), -6, -9.29624353850725),
 ((1, 3), 1, (2, 3), -10, -9.219757102548096),
 ((2, 3), 1, (3, 3), -14, -5.799942413340491),
 ((3, 3), 3, (3, 4), -16, -1.9999998004122235),
 ((3, 4), 1, (4, 4), -16, 0.0)]